# 소득 예측 AI 해커톤
- `알고리즘`, `정형`, `회귀`, `사회`, `RMSE`
- 기간
    - 2024.03.11 ~ 2024.04.08 09:59
- https://dacon.io/competitions/official/236230/data

## 배경

> 안녕하세요!
>
> 해커톤 41회, 소득 예측 AI 해커톤에 오신 것을 환영합니다.  
> 
> 이번 데이콘 해커톤은 다양한 개인적 특성을 바탕으로 한 데이터를 활용하여 소득 수준을 예측하는 것을 목표로 합니다.
> 
> 참가자 여러분들은 제공된 개인 특성 데이터를 기반으로 한 알고리즘 개발을 통해 
> 
> 개인의 소득 수준을 예측함으로써 사회적 문제 해결에 기여할 수 있는 기회가 될 것입니다. 
> 
> 또한, 이번 대회의 목표는 단순히 개인 소득 수준을 예측하는 것을 넘어, 
> 
> 다양한 개인적 특성 데이터를 분석하고 이해하는 능력을 키우며,
> 
> 이를 통해 AI 역량과 경험을 한 단계 더 발전시키는 것입니다. 




## 주제
- 개인 특성 데이터를 활용하여 개인 소득 수준을 예측하는 AI 모델 개발

## 설명
- 개인의 특성과 관련된 다양한 데이터를 활용하여 개인 소득 수준을 예측할 수 있는 AI 모델 개발

### 컬럼 설명
- `train.csv`
    - 한 사람에 관련된 다양한 사회적, 경제적 정보
    - `ID`: 학습 데이터 고유 ID
    - `Age` : 나이
    - `Gender` : 성별
    - `Education_Status` : 교육 수준
    - `Employment_Status` : 고용 형태
    - `Working_Week (Yearly)` : 연간 근무하는 주
    - `Industry_Status` : 고용 분야
    - `Occupation_Status` : 직업 형태
    - `Race` : 인종
    - `Hispanic_Origin` : 히스패닉 혈통
    - `Martial_Status` : 결혼 유무
    - `Household_Status` : 가정 세대원 상태
    - `Household_Summary` : 가정 세대원 상태 요약
    - `Citizenship` : 시민권
    - `Birth_Country` : 출생국가
    - `Birth_Country (Father)`
    - `Birth_Country (Mother)`
    - `Tax_Status` : 세금 형태
    - `Gains` : 소득 금액
    - `Losses` : 손실 금액
    - `Dividends` : 배당 금액
    - `Income_Status` : 최종 소득 상태
    - `Income` : 예측 목표, 1시간 단위의 소득을 예측 (0일 경우 근로소득이 없다)
- `test.csv`
    - 한 사람에 관련된 다양한 사회적, 경제적 정보
    - `ID` : 테스트 데이터 고유 ID
    - Income이 존재하지 않음
- `sample_submission.csv`
    - `ID` : 테스트 데이터 고유 ID
    - `Income` : ID에 해당되는 Income을 예측하여 제출

# 데이터셋 전처리

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

pd.set_option('display.max_columns', None) # 데이터프레임 모든 열 출력

from datetime import datetime
from datetime import timedelta
import missingno as msno

In [61]:
train_path = "C:/Users/aryij/Documents/DataStudy/income-prediction-dacon/data/train.csv"
test_path = "C:/Users/aryij/Documents/DataStudy/income-prediction-dacon/data/test.csv"
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)
train_df.head()

,ID,Age,Gender,Education_Status,Employment_Status,Working_Week (Yearly),Industry_Status,Occupation_Status,Race,Hispanic_Origin,Martial_Status,Household_Status,Household_Summary,Citizenship,Birth_Country,Birth_Country (Father),Birth_Country (Mother),Tax_Status,Gains,Losses,Dividends,Income_Status,Income
0,TRAIN_00000,63,M,Middle (7-8),Full-Time,4,Social Services,Services,White,All other,Married,Householder,Householder,Native,US,US,US,Nonfiler,0,0,0,Unknown,425
1,TRAIN_00001,37,M,Associates degree (Vocational),Full-Time,52,Entertainment,Services,White,All other,Separated,Nonfamily householder,Householder,Native,US,US,US,Single,0,0,0,Under Median,0
2,TRAIN_00002,58,F,High graduate,Full-Time,52,Manufacturing (Non-durable),Admin Support (include Clerical),Black,All other,Married,Householder,Householder,Native,US,US,US,Married Filling Jointly both under 65 (MFJ),3411,0,0,Under Median,860
3,TRAIN_00003,44,M,High graduate,Full-Time,52,Retail,Technicians & Support,White,All other,Divorced,Nonfamily householder,Householder,Native,US,US,US,Single,0,0,0,Under Median,850
4,TRAIN_00004,37,F,High graduate,Full-Time,52,Retail,Sales,White,All other,Divorced,Householder,Householder,Native,US,US,US,Head of Household (HOH),0,0,0,Unknown,570


In [67]:
total_df = pd.concat([train_df, test_df])
total_df.head()

,ID,Age,Gender,Education_Status,Employment_Status,Working_Week (Yearly),Industry_Status,Occupation_Status,Race,Hispanic_Origin,Martial_Status,Household_Status,Household_Summary,Citizenship,Birth_Country,Birth_Country (Father),Birth_Country (Mother),Tax_Status,Gains,Losses,Dividends,Income_Status,Income
0,TRAIN_00000,63,M,Middle (7-8),Full-Time,4,Social Services,Services,White,All other,Married,Householder,Householder,Native,US,US,US,Nonfiler,0,0,0,Unknown,425.0
1,TRAIN_00001,37,M,Associates degree (Vocational),Full-Time,52,Entertainment,Services,White,All other,Separated,Nonfamily householder,Householder,Native,US,US,US,Single,0,0,0,Under Median,0.0
2,TRAIN_00002,58,F,High graduate,Full-Time,52,Manufacturing (Non-durable),Admin Support (include Clerical),Black,All other,Married,Householder,Householder,Native,US,US,US,Married Filling Jointly both under 65 (MFJ),3411,0,0,Under Median,860.0
3,TRAIN_00003,44,M,High graduate,Full-Time,52,Retail,Technicians & Support,White,All other,Divorced,Nonfamily householder,Householder,Native,US,US,US,Single,0,0,0,Under Median,850.0
4,TRAIN_00004,37,F,High graduate,Full-Time,52,Retail,Sales,White,All other,Divorced,Householder,Householder,Native,US,US,US,Head of Household (HOH),0,0,0,Unknown,570.0


In [68]:
total_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30000 entries, 0 to 9999
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID                      30000 non-null  object 
 1   Age                     30000 non-null  int64  
 2   Gender                  30000 non-null  object 
 3   Education_Status        30000 non-null  object 
 4   Employment_Status       30000 non-null  object 
 5   Working_Week (Yearly)   30000 non-null  int64  
 6   Industry_Status         30000 non-null  object 
 7   Occupation_Status       30000 non-null  object 
 8   Race                    30000 non-null  object 
 9   Hispanic_Origin         30000 non-null  object 
 10  Martial_Status          30000 non-null  object 
 11  Household_Status        29999 non-null  object 
 12  Household_Summary       30000 non-null  object 
 13  Citizenship             30000 non-null  object 
 14  Birth_Country           30000 non-null  obje

In [3]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   ID                      20000 non-null  object
 1   Age                     20000 non-null  int64 
 2   Gender                  20000 non-null  object
 3   Education_Status        20000 non-null  object
 4   Employment_Status       20000 non-null  object
 5   Working_Week (Yearly)   20000 non-null  int64 
 6   Industry_Status         20000 non-null  object
 7   Occupation_Status       20000 non-null  object
 8   Race                    20000 non-null  object
 9   Hispanic_Origin         20000 non-null  object
 10  Martial_Status          20000 non-null  object
 11  Household_Status        20000 non-null  object
 12  Household_Summary       20000 non-null  object
 13  Citizenship             20000 non-null  object
 14  Birth_Country           20000 non-null  object
 15  Bi

In [4]:
# 결측치 확인 -> 결측치 없음
train_df.isnull().sum()

ID                        0
Age                       0
Gender                    0
Education_Status          0
Employment_Status         0
Working_Week (Yearly)     0
Industry_Status           0
Occupation_Status         0
Race                      0
Hispanic_Origin           0
Martial_Status            0
Household_Status          0
Household_Summary         0
Citizenship               0
Birth_Country             0
Birth_Country (Father)    0
Birth_Country (Mother)    0
Tax_Status                0
Gains                     0
Losses                    0
Dividends                 0
Income_Status             0
Income                    0
dtype: int64

## `describe()` 결과

### 수치형 변수
- `Working_Week (Yearly)` : 데이터의 50% 부터 최대치까지 전부 52주 근무
    - → 일반적으로 52주 근무 하는 사람들이 많다
- `Gains` : 75%까지 0인 것에 비해 평균 약 383, 최대치 99999, 표준편차 약 4144
    - → 소수의 인원이 높은 소득금액을 벌어 들인다
- `Losses` : 75%까지 0인 것에 비해 평균 약 40, 최대치 4356, 표준편차 279
    - → 극소수의 인원이 높은 손실 금액을 기록했다
- `Dividends` : 75%까지 0인 것에 비해 평균 약 123, 최대치 45000, 표준편차 약 4144
    - → 극소수의 인원이 높은 배당 금액 기록했다
- `Income` : 50%가 500, 평균 약 555, 중위값 근처에서 평균값이 형성됨
    - → 1시간 단위의 소득이 중위값 500을 조금 넘는 사람들이 많다

In [5]:
train_df.describe()

,Age,Working_Week (Yearly),Gains,Losses,Dividends,Income
count,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000
mean,35.632500,34.943050,383.129500,40.202150,123.451450,554.565250
std,17.994414,22.254592,4144.247487,279.182677,1206.949429,701.553155
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,23.000000,7.000000,0.000000,0.000000,0.000000,0.000000
50%,34.000000,52.000000,0.000000,0.000000,0.000000,500.000000
75%,47.000000,52.000000,0.000000,0.000000,0.000000,875.000000
max,90.000000,52.000000,99999.000000,4356.000000,45000.000000,9999.000000


### 범주형 변수
- `Gender` : 여성이 10472, 남성이 9528 (약 52:48 비율)
- `Education_Status` : High graduate (고졸) 학력이 제일 많다
- `Employment_Status` : 아이 또는 군인이 가장 많다
- `Industry_Status` : 대학을 가지 않거나 아이인 경우가 가장 많다
- `Occupation_Status` : Unknown이 가장 많다
- `Race` : 백인이 가장 많다
- `Hispanic_Origin` : All other가 가장 많다
- `Martial_Status` : Married가 가장 많음
- `Household_Status` : Householder (세대주)인 경우가 가장 많다
- `Household_Summary` : Householder (세대주)인 경우가 가장 많다
- `Citizenship` : Native가 가장 많다
- `Birth_Country` : US (미국)가 가장 많다
- `Birth_Country (Father)` : US (미국)가 가장 많다
- `Birth_Country (Mother)` : US (미국)가 가장 많다
- `Tax_Status` : Married Filing Jointly (부부 합산 신고)가 가장 많다
    - 기혼인 부부가 각 소득을 합산하여 세금 보고
    - 가장 낮은 세율과 많은 세제 혜택이 있음
- `Income_Status` : Under Median (중위수 이하)가 가장 많다

In [6]:
train_df.describe(include="object")

,ID,Gender,Education_Status,Employment_Status,Industry_Status,Occupation_Status,Race,Hispanic_Origin,Martial_Status,Household_Status,Household_Summary,Citizenship,Birth_Country,Birth_Country (Father),Birth_Country (Mother),Tax_Status,Income_Status
count,20000,20000,20000,20000,20000,20000,20000,20000,20000,20000,20000,20000,20000,20000,20000,20000,20000
unique,20000,2,17,8,24,15,5,10,7,31,8,5,43,43,43,6,3
top,TRAIN_00000,F,High graduate,Children or Armed Forces,Not in universe or children,Unknown,White,All other,Married,Householder,Householder,Native,US,US,US,Married Filling Jointly both under 65 (MFJ),Under Median
freq,1,10472,6494,11142,4688,4688,16845,17769,9554,6087,8552,17825,17825,16563,16594,8588,13237


In [7]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   ID                      20000 non-null  object
 1   Age                     20000 non-null  int64 
 2   Gender                  20000 non-null  object
 3   Education_Status        20000 non-null  object
 4   Employment_Status       20000 non-null  object
 5   Working_Week (Yearly)   20000 non-null  int64 
 6   Industry_Status         20000 non-null  object
 7   Occupation_Status       20000 non-null  object
 8   Race                    20000 non-null  object
 9   Hispanic_Origin         20000 non-null  object
 10  Martial_Status          20000 non-null  object
 11  Household_Status        20000 non-null  object
 12  Household_Summary       20000 non-null  object
 13  Citizenship             20000 non-null  object
 14  Birth_Country           20000 non-null  object
 15  Bi

## 컬럼명 변경

In [8]:
# 데이터프레임 컬럼명 소문자로 변경하는 함수 생성
def lower_column(df):
    lower_cols = []

    # .lower() 사용하여 대문자로 적힌 컬럼명 소문자로 변경
    for column in train_df.columns:
        lower_cols.append(column.lower())
        
    # 소문자로 변경한 컬럼명을 새롭게 데이터프레임의 컬럼명으로 지정
    df.columns = lower_cols
    return df.columns

In [9]:
# 변경한 소문자 함수 적용
lower_column(train_df)

Index(['id', 'age', 'gender', 'education_status', 'employment_status',
       'working_week (yearly)', 'industry_status', 'occupation_status', 'race',
       'hispanic_origin', 'martial_status', 'household_status',
       'household_summary', 'citizenship', 'birth_country',
       'birth_country (father)', 'birth_country (mother)', 'tax_status',
       'gains', 'losses', 'dividends', 'income_status', 'income'],
      dtype='object')

In [10]:
# 변경한 내역 확인
train_df.head()

,id,age,gender,education_status,employment_status,working_week (yearly),industry_status,occupation_status,race,hispanic_origin,martial_status,household_status,household_summary,citizenship,birth_country,birth_country (father),birth_country (mother),tax_status,gains,losses,dividends,income_status,income
0,TRAIN_00000,63,M,Middle (7-8),Full-Time,4,Social Services,Services,White,All other,Married,Householder,Householder,Native,US,US,US,Nonfiler,0,0,0,Unknown,425
1,TRAIN_00001,37,M,Associates degree (Vocational),Full-Time,52,Entertainment,Services,White,All other,Separated,Nonfamily householder,Householder,Native,US,US,US,Single,0,0,0,Under Median,0
2,TRAIN_00002,58,F,High graduate,Full-Time,52,Manufacturing (Non-durable),Admin Support (include Clerical),Black,All other,Married,Householder,Householder,Native,US,US,US,Married Filling Jointly both under 65 (MFJ),3411,0,0,Under Median,860
3,TRAIN_00003,44,M,High graduate,Full-Time,52,Retail,Technicians & Support,White,All other,Divorced,Nonfamily householder,Householder,Native,US,US,US,Single,0,0,0,Under Median,850
4,TRAIN_00004,37,F,High graduate,Full-Time,52,Retail,Sales,White,All other,Divorced,Householder,Householder,Native,US,US,US,Head of Household (HOH),0,0,0,Unknown,570


## 컬럼별 확인

### `household_status`, `household_summary`
- `household_summary` 통해 `household_status` 요약 알 수 있다 → `household_status` 컬럼 삭제

In [11]:
train_df.household_status.value_counts()

household_status
Householder                                                               6087
Spouse of householder                                                     4794
Child <18 never marr not in subfamily                                     2670
Nonfamily householder                                                     2465
Child 18+ never marr Not in a subfamily                                   1860
Secondary individual                                                       845
Other Rel 18+ never marr not in subfamily                                  195
Other Rel 18+ ever marr not in subfamily                                   154
Child 18+ ever marr Not in a subfamily                                     118
Child 18+ ever married Responsible Person of subfamily                     101
Child 18+ never married Responsible Person of subfamily                     96
Grandchild <18 never married child of subfamily Responsible Person          88
Responsible Person of unrelated sub

In [12]:
train_df.household_summary.value_counts()

household_summary
Householder                             8552
Spouse of householder                   4794
Child under 18 never married            2679
Child 18 or older                       2192
Nonrelative of householder               974
Other relative of householder            781
Group Quarters- Secondary individual      24
Child under 18 ever married                4
Name: count, dtype: int64

### `occupation_status`
- `unknown`이 가장 많다
    - `unknown` 그대로 두면 결측치와 다를바가 없지 않을까? → 최대한 대체 시도

In [13]:
# occupation_status : Unknown이 가장 많다
train_df.occupation_status.value_counts()

occupation_status
Unknown                             4688
Admin Support (include Clerical)    2709
Services                            2313
Craft & Repair                      1869
Sales                               1692
Professional                        1488
Machine Operators & Inspectors      1383
Management                          1111
Handlers/Cleaners                    837
Transportation                       690
Technicians & Support                558
Farming & Forestry & Fishing         296
Protective Services                  260
Private Household Services           105
Armed Forces                           1
Name: count, dtype: int64

In [14]:
# occupation_status Unknown인 것들만 모아서 확인
train_df[train_df.occupation_status=="Unknown"].head()

,id,age,gender,education_status,employment_status,working_week (yearly),industry_status,occupation_status,race,hispanic_origin,martial_status,household_status,household_summary,citizenship,birth_country,birth_country (father),birth_country (mother),tax_status,gains,losses,dividends,income_status,income
6,TRAIN_00006,67,M,Middle (7-8),Children or Armed Forces,0,Not in universe or children,Unknown,White,All other,Divorced,Nonfamily householder,Householder,Native,US,US,US,Nonfiler,0,0,0,Unknown,0
7,TRAIN_00007,64,M,Masters degree,Not Working,5,Not in universe or children,Unknown,White,All other,Married,Householder,Householder,Native,US,US,US,Married Filling Jointly both under 65 (MFJ),0,0,2052,Under Median,0
11,TRAIN_00011,75,F,High Freshman,Children or Armed Forces,0,Not in universe or children,Unknown,White,Cuban,Married,Other Relative 18+ ever married Responsible Pe...,Other relative of householder,Foreign-born (Non-US Citizen),Cuba,Cuba,Cuba,Nonfiler,0,0,0,Unknown,0
16,TRAIN_00016,10,M,Children,Children or Armed Forces,0,Not in universe or children,Unknown,White,All other,Single,Child <18 never marr not in subfamily,Child under 18 never married,Native,US,US,US,Nonfiler,0,0,0,Unknown,0
21,TRAIN_00021,16,F,High Freshman,Not Working,14,Not in universe or children,Unknown,White,All other,Single,Child <18 never marr not in subfamily,Child under 18 never married,Native,US,US,US,Single,0,0,0,Under Median,0


In [15]:
# 전체 데이터에서  occupation_status Unknown인 사람의 비율
print("occupation_status가 Unknown인 사람의 비율 :", train_df[train_df.occupation_status=="Unknown"].shape[0] / train_df.shape[0] * 100, "%")

occupation_status가 Unknown인 사람의 비율 : 23.44 %


In [16]:
# occupation_status가 Unknown인 경우 industry_status : 모두 Not in universe or children
train_df[train_df.occupation_status=="Unknown"]["industry_status"].value_counts()

industry_status
Not in universe or children    4688
Name: count, dtype: int64

In [17]:
# 반대로 industry_status가 Not in universe or children인 경우 occupation_status : 모두 Unknown
train_df[train_df.industry_status=="Not in universe or children"]["occupation_status"].value_counts()

occupation_status
Unknown    4688
Name: count, dtype: int64

In [18]:
# occupation_status가 Unknown인 것 중 age 다양함
train_df[train_df.occupation_status=="Unknown"]["age"].value_counts()

age
14    179
7     168
1     164
6     154
8     147
     ... 
49     15
52     13
87     11
89     10
88      8
Name: count, Length: 91, dtype: int64

In [19]:
# 미국 미성년자 기준 : 18세 미만
# 18세 미만 중 occupation_status가 Unknown인 것 확인
# 18세 미만인데  employment_status가 Children or Armed Forces
# - employment_status를 Children으로 바꿔도 될 듯
# - industry_status도 children으로 변경
# occupation_status이 Unknown이면 children으로 변경 (아니면 student로 변경?)

train_df[(train_df.age<18) & (train_df.occupation_status=="Unknown")].head()

,id,age,gender,education_status,employment_status,working_week (yearly),industry_status,occupation_status,race,hispanic_origin,martial_status,household_status,household_summary,citizenship,birth_country,birth_country (father),birth_country (mother),tax_status,gains,losses,dividends,income_status,income
16,TRAIN_00016,10,M,Children,Children or Armed Forces,0,Not in universe or children,Unknown,White,All other,Single,Child <18 never marr not in subfamily,Child under 18 never married,Native,US,US,US,Nonfiler,0,0,0,Unknown,0
21,TRAIN_00021,16,F,High Freshman,Not Working,14,Not in universe or children,Unknown,White,All other,Single,Child <18 never marr not in subfamily,Child under 18 never married,Native,US,US,US,Single,0,0,0,Under Median,0
35,TRAIN_00035,16,F,High Freshman,Not Working,0,Not in universe or children,Unknown,White,Puerto Rican,Single,Child <18 never marr not in subfamily,Child under 18 never married,Native,US,Puerto-Rico,Puerto-Rico,Nonfiler,0,0,0,Under Median,0
41,TRAIN_00041,7,M,Children,Children or Armed Forces,0,Not in universe or children,Unknown,White,All other,Single,Child <18 never marr not in subfamily,Child under 18 never married,Native,US,US,US,Nonfiler,0,0,0,Under Median,0
56,TRAIN_00056,17,F,High Freshman,Not Working,0,Not in universe or children,Unknown,White,All other,Single,Secondary individual,Nonrelative of householder,Native,US,US,US,Nonfiler,0,0,0,Under Median,0


In [20]:
train_df[(train_df.age<18) & (train_df.occupation_status=="Unknown")].shape

(2464, 23)

In [21]:
train_df[(train_df.age<18) & (train_df.occupation_status=="Unknown")]["employment_status"].value_counts()

employment_status
Children or Armed Forces    2323
Not Working                  136
Seeking Part-Time              5
Name: count, dtype: int64

In [22]:
train_df[(train_df.age<18) & (train_df.occupation_status=="Unknown")]["industry_status"].value_counts()

industry_status
Not in universe or children    2464
Name: count, dtype: int64

In [23]:
train_df[(train_df.age<18) & (train_df.industry_status=="Not in universe or children")].shape

(2464, 23)

In [24]:
train_df[train_df.industry_status=="Not in universe or children"]

,id,age,gender,education_status,employment_status,working_week (yearly),industry_status,occupation_status,race,hispanic_origin,martial_status,household_status,household_summary,citizenship,birth_country,birth_country (father),birth_country (mother),tax_status,gains,losses,dividends,income_status,income
6,TRAIN_00006,67,M,Middle (7-8),Children or Armed Forces,0,Not in universe or children,Unknown,White,All other,Divorced,Nonfamily householder,Householder,Native,US,US,US,Nonfiler,0,0,0,Unknown,0
7,TRAIN_00007,64,M,Masters degree,Not Working,5,Not in universe or children,Unknown,White,All other,Married,Householder,Householder,Native,US,US,US,Married Filling Jointly both under 65 (MFJ),0,0,2052,Under Median,0
11,TRAIN_00011,75,F,High Freshman,Children or Armed Forces,0,Not in universe or children,Unknown,White,Cuban,Married,Other Relative 18+ ever married Responsible Pe...,Other relative of householder,Foreign-born (Non-US Citizen),Cuba,Cuba,Cuba,Nonfiler,0,0,0,Unknown,0
16,TRAIN_00016,10,M,Children,Children or Armed Forces,0,Not in universe or children,Unknown,White,All other,Single,Child <18 never marr not in subfamily,Child under 18 never married,Native,US,US,US,Nonfiler,0,0,0,Unknown,0
21,TRAIN_00021,16,F,High Freshman,Not Working,14,Not in universe or children,Unknown,White,All other,Single,Child <18 never marr not in subfamily,Child under 18 never married,Native,US,US,US,Single,0,0,0,Under Median,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19986,TRAIN_19986,31,M,Middle (7-8),Not Working,0,Not in universe or children,Unknown,White,All other,Divorced,Householder,Householder,Native,US,US,US,Nonfiler,0,0,0,Unknown,0
19989,TRAIN_19989,14,M,Children,Children or Armed Forces,0,Not in universe or children,Unknown,White,All other,Single,Child <18 never marr not in subfamily,Child under 18 never married,Native,US,US,US,Nonfiler,0,0,0,Unknown,0
19990,TRAIN_19990,8,M,Children,Children or Armed Forces,0,Not in universe or children,Unknown,Asian/Pacific,All other,Single,Child <18 never marr not in subfamily,Child under 18 never married,Foreign-born (Non-US Citizen),Japan,Japan,Japan,Nonfiler,0,0,0,Unknown,0
19994,TRAIN_19994,17,F,High Sophomore,Not Working,0,Not in universe or children,Unknown,White,All other,Single,Child <18 never marr not in subfamily,Child under 18 never married,Native,US,US,US,Nonfiler,0,0,0,Under Median,0


In [25]:
# 18세 미만이지만 occupation_status가 Unknown이 아닌 사람들
train_df[(train_df.age<18) & (train_df.occupation_status!="Unknown")].head()

,id,age,gender,education_status,employment_status,working_week (yearly),industry_status,occupation_status,race,hispanic_origin,martial_status,household_status,household_summary,citizenship,birth_country,birth_country (father),birth_country (mother),tax_status,gains,losses,dividends,income_status,income
29,TRAIN_00029,15,F,Middle (7-8),Children or Armed Forces,12,Social Services,Services,White,All other,Single,Child <18 never marr not in subfamily,Child under 18 never married,Native,US,US,US,Nonfiler,0,0,0,Under Median,0
38,TRAIN_00038,17,F,High Junior,Children or Armed Forces,52,Medical (except Hospitals),Services,White,All other,Single,Other Relative <18 never married child of subf...,Other relative of householder,Native,US,US,US,Single,0,0,0,Unknown,0
48,TRAIN_00048,17,M,High Freshman,Children or Armed Forces,26,Retail,Handlers/Cleaners,White,All other,Single,Child <18 never marr not in subfamily,Child under 18 never married,Native,US,US,US,Single,0,0,0,Under Median,550
89,TRAIN_00089,15,F,High Freshman,Children or Armed Forces,6,Private Household Services,Private Household Services,White,All other,Single,Child <18 never marr not in subfamily,Child under 18 never married,Native,US,US,Unknown,Nonfiler,0,0,0,Under Median,200
127,TRAIN_00127,16,F,High Sophomore,Children or Armed Forces,20,Private Household Services,Private Household Services,White,All other,Single,Child <18 never marr not in subfamily,Child under 18 never married,Native,US,US,US,Nonfiler,0,0,0,Under Median,435


In [26]:
# 18세 이상 (성인)인데 occupation_status가 Unknown인 사람들 (2224먕)
train_df[(train_df.age>=18) & (train_df.occupation_status=="Unknown")].head()

,id,age,gender,education_status,employment_status,working_week (yearly),industry_status,occupation_status,race,hispanic_origin,martial_status,household_status,household_summary,citizenship,birth_country,birth_country (father),birth_country (mother),tax_status,gains,losses,dividends,income_status,income
6,TRAIN_00006,67,M,Middle (7-8),Children or Armed Forces,0,Not in universe or children,Unknown,White,All other,Divorced,Nonfamily householder,Householder,Native,US,US,US,Nonfiler,0,0,0,Unknown,0
7,TRAIN_00007,64,M,Masters degree,Not Working,5,Not in universe or children,Unknown,White,All other,Married,Householder,Householder,Native,US,US,US,Married Filling Jointly both under 65 (MFJ),0,0,2052,Under Median,0
11,TRAIN_00011,75,F,High Freshman,Children or Armed Forces,0,Not in universe or children,Unknown,White,Cuban,Married,Other Relative 18+ ever married Responsible Pe...,Other relative of householder,Foreign-born (Non-US Citizen),Cuba,Cuba,Cuba,Nonfiler,0,0,0,Unknown,0
22,TRAIN_00022,31,F,High Sophomore,Children or Armed Forces,0,Not in universe or children,Unknown,White,Puerto Rican,Married (Spouse Absent),Householder,Householder,Native (Born in Puerto Rico or US Outlying),Puerto-Rico,Puerto-Rico,Puerto-Rico,Nonfiler,0,0,0,Unknown,0
23,TRAIN_00023,76,M,High Sophomore,Children or Armed Forces,0,Not in universe or children,Unknown,White,All other,Married,Householder,Householder,Foreign-born (Naturalized US Citizen),Germany,Germany,Germany,Married Filling Jointly both over 65 (MFJ),0,0,0,Under Median,0


In [27]:
train_df[(train_df.age>=18) & (train_df.occupation_status=="Unknown") & (train_df.industry_status!="Not in universe or children")]

,id,age,gender,education_status,employment_status,working_week (yearly),industry_status,occupation_status,race,hispanic_origin,martial_status,household_status,household_summary,citizenship,birth_country,birth_country (father),birth_country (mother),tax_status,gains,losses,dividends,income_status,income


In [28]:
# 미국 군인 (준장 이하) 일반적으로 62세 정년
# 이 사람들 중 employment_status이 Children or Armed Forces	인 사람들은 어디에도 해당하지 않음
# employment_status → Not Working으로 변경
train_df[(train_df.age>=62) & (train_df.occupation_status=="Unknown")].head()

,id,age,gender,education_status,employment_status,working_week (yearly),industry_status,occupation_status,race,hispanic_origin,martial_status,household_status,household_summary,citizenship,birth_country,birth_country (father),birth_country (mother),tax_status,gains,losses,dividends,income_status,income
6,TRAIN_00006,67,M,Middle (7-8),Children or Armed Forces,0,Not in universe or children,Unknown,White,All other,Divorced,Nonfamily householder,Householder,Native,US,US,US,Nonfiler,0,0,0,Unknown,0
7,TRAIN_00007,64,M,Masters degree,Not Working,5,Not in universe or children,Unknown,White,All other,Married,Householder,Householder,Native,US,US,US,Married Filling Jointly both under 65 (MFJ),0,0,2052,Under Median,0
11,TRAIN_00011,75,F,High Freshman,Children or Armed Forces,0,Not in universe or children,Unknown,White,Cuban,Married,Other Relative 18+ ever married Responsible Pe...,Other relative of householder,Foreign-born (Non-US Citizen),Cuba,Cuba,Cuba,Nonfiler,0,0,0,Unknown,0
23,TRAIN_00023,76,M,High Sophomore,Children or Armed Forces,0,Not in universe or children,Unknown,White,All other,Married,Householder,Householder,Foreign-born (Naturalized US Citizen),Germany,Germany,Germany,Married Filling Jointly both over 65 (MFJ),0,0,0,Under Median,0
28,TRAIN_00028,63,M,Masters degree,Not Working,6,Not in universe or children,Unknown,White,All other,Married,Householder,Householder,Native,US,US,US,Married Filling Jointly both under 65 (MFJ),0,0,0,Unknown,0


In [29]:
train_df.occupation_status.value_counts()

occupation_status
Unknown                             4688
Admin Support (include Clerical)    2709
Services                            2313
Craft & Repair                      1869
Sales                               1692
Professional                        1488
Machine Operators & Inspectors      1383
Management                          1111
Handlers/Cleaners                    837
Transportation                       690
Technicians & Support                558
Farming & Forestry & Fishing         296
Protective Services                  260
Private Household Services           105
Armed Forces                           1
Name: count, dtype: int64

In [30]:
train_df[train_df.occupation_status=="Armed Forces"]

,id,age,gender,education_status,employment_status,working_week (yearly),industry_status,occupation_status,race,hispanic_origin,martial_status,household_status,household_summary,citizenship,birth_country,birth_country (father),birth_country (mother),tax_status,gains,losses,dividends,income_status,income
10342,TRAIN_10342,31,F,College,Seeking Full-Time,20,Armed Forces,Armed Forces,White,Puerto Rican,Married,Spouse of householder,Spouse of householder,Native,US,Puerto-Rico,Puerto-Rico,Married Filling Jointly both under 65 (MFJ),0,0,0,Under Median,0


In [31]:
train_df.employment_status.value_counts()

employment_status
Children or Armed Forces         11142
Full-Time                         6588
Not Working                       1221
Choice Part-Time                   593
Part-Time (Usually Part-Time)      158
Seeking Full-Time                  121
Part-Time (Usually Full-Time)      100
Seeking Part-Time                   77
Name: count, dtype: int64

In [32]:
train_df.industry_status.value_counts()

industry_status
Not in universe or children                     4688
Retail                                          3149
Manufacturing (Durable)                         1575
Manufacturing (Non-durable)                     1223
Education                                       1041
Business & Repair                                847
Medical (except Hospitals)                       838
Construction                                     832
Hospitals                                        821
Finance Insurance & Real Estate                  727
Transportation                                   693
Public Administration                            641
Other professional services                      477
Wholesale                                        450
Personal Services (except Private Household)     429
Social Services                                  367
Entertainment                                    278
Agriculture                                      268
Utilities & Sanitary          

In [33]:
# 18세 이상 62세 미만인데 occupation_status가 Unknown
# - industry_status : 전부 Not in universe or children
# - income : 전부 0
# - working_week (yearly) : 0이 많긴해도 다양함
# employment_status가 Children or Armed Forces 인 사람들
# - employment_status, occupation_status : Not Working으로 변경

train_df[(train_df.age>=18) 
         & (train_df.age<62) 
         & (train_df.occupation_status=="Unknown")
         & (train_df.employment_status=="Children or Armed Forces")].head()

,id,age,gender,education_status,employment_status,working_week (yearly),industry_status,occupation_status,race,hispanic_origin,martial_status,household_status,household_summary,citizenship,birth_country,birth_country (father),birth_country (mother),tax_status,gains,losses,dividends,income_status,income
22,TRAIN_00022,31,F,High Sophomore,Children or Armed Forces,0,Not in universe or children,Unknown,White,Puerto Rican,Married (Spouse Absent),Householder,Householder,Native (Born in Puerto Rico or US Outlying),Puerto-Rico,Puerto-Rico,Puerto-Rico,Nonfiler,0,0,0,Unknown,0
39,TRAIN_00039,18,M,High Junior,Children or Armed Forces,0,Not in universe or children,Unknown,White,All other,Single,Child 18+ never marr Not in a subfamily,Child 18 or older,Native,US,US,US,Nonfiler,0,0,0,Under Median,0
55,TRAIN_00055,61,M,Middle (7-8),Children or Armed Forces,0,Not in universe or children,Unknown,Black,All other,Married,Householder,Householder,Native,US,US,US,Married Filling Jointly both under 65 (MFJ),0,0,0,Under Median,0
62,TRAIN_00062,61,F,High graduate,Children or Armed Forces,0,Not in universe or children,Unknown,White,All other,Married,Spouse of householder,Spouse of householder,Native,US,US,US,Married Filling Jointly both under 65 (MFJ),0,0,0,Unknown,0
155,TRAIN_00155,23,M,High graduate,Children or Armed Forces,0,Not in universe or children,Unknown,White,All other,Single,Nonfamily householder,Householder,Native,US,US,US,Single,0,0,0,Unknown,0


In [34]:
# 18세 이상 62세 미만, occupation_status가 Unknown이 아니고, employment_status가 children or armed forces
# → employment_status : Armed Forces로 변경
train_df[(train_df.age>=18) 
         & (train_df.age<62) 
         & (train_df.occupation_status!="Unknown")
         & (train_df.employment_status=="Children or Armed Forces")].head()

,id,age,gender,education_status,employment_status,working_week (yearly),industry_status,occupation_status,race,hispanic_origin,martial_status,household_status,household_summary,citizenship,birth_country,birth_country (father),birth_country (mother),tax_status,gains,losses,dividends,income_status,income
8,TRAIN_00008,24,F,Bachelors degree,Children or Armed Forces,52,Retail,Sales,White,All other,Single,Child 18+ never marr Not in a subfamily,Child 18 or older,Native,US,US,US,Single,0,0,0,Under Median,0
10,TRAIN_00010,56,F,High graduate,Children or Armed Forces,52,Hospitals,Services,Black,All other,Married,Spouse of householder,Spouse of householder,Native,US,US,US,Married Filling Jointly one over 65 & one unde...,0,0,0,Under Median,0
14,TRAIN_00014,44,M,High graduate,Children or Armed Forces,52,Public Administration,Protective Services,White,All other,Married,Householder,Householder,Native,US,US,US,Married Filling Jointly both under 65 (MFJ),0,0,200,Over Median,0
17,TRAIN_00017,48,F,Bachelors degree,Children or Armed Forces,52,Finance Insurance & Real Estate,Management,White,All other,Divorced,Nonfamily householder,Householder,Native,US,US,US,Single,0,0,0,Under Median,850
19,TRAIN_00019,29,F,College,Children or Armed Forces,52,Finance Insurance & Real Estate,Admin Support (include Clerical),White,All other,Married,Householder,Householder,Native,US,US,US,Married Filling Jointly both under 65 (MFJ),0,0,0,Unknown,950


In [35]:
# occupation_status가 Unknown인 경우, income 또한 전부 0
train_df[train_df.occupation_status=="Unknown"]["income"].value_counts()

income
0    4688
Name: count, dtype: int64

In [36]:
train_df[train_df.occupation_status=="Armed Forces"]

,id,age,gender,education_status,employment_status,working_week (yearly),industry_status,occupation_status,race,hispanic_origin,martial_status,household_status,household_summary,citizenship,birth_country,birth_country (father),birth_country (mother),tax_status,gains,losses,dividends,income_status,income
10342,TRAIN_10342,31,F,College,Seeking Full-Time,20,Armed Forces,Armed Forces,White,Puerto Rican,Married,Spouse of householder,Spouse of householder,Native,US,Puerto-Rico,Puerto-Rico,Married Filling Jointly both under 65 (MFJ),0,0,0,Under Median,0


### income_status
- Income의 경우 1시간 단위의 근로 소득을 의미하고, Income_Status의 경우 최종 소득을 바탕으로 구성되어 있다
- https://dacon.io/competitions/official/236230/talkboard/411247?page=1&dtype=recent

In [37]:
# Unknown 6026개 (많다)
train_df["income_status"].value_counts()

income_status
Under Median    13237
Unknown          6026
Over Median       737
Name: count, dtype: int64

In [38]:
print("income_status 컬럼의 Unknown 비율 :", round((train_df["income_status"].value_counts()["Unknown"] / train_df.shape[0])*100, 2), "%")

income_status 컬럼의 Unknown 비율 : 30.13 %


In [39]:
# Unknown을 최대한 대체
# Under Median의 가장 높은 값 구해본다

In [40]:
# income 컬럼의 중위값 : 500
np.median(train_df["income"])

500.0

In [41]:
train_df[train_df.income_status=="Under Median"]["income"].value_counts()

income
0       5674
500      520
700      382
600      382
800      346
        ... 
534        1
2375       1
1602       1
1194       1
2050       1
Name: count, Length: 958, dtype: int64

In [42]:
train_df[train_df.income_status=="Under Median"]["income"].value_counts()

income
0       5674
500      520
700      382
600      382
800      346
        ... 
534        1
2375       1
1602       1
1194       1
2050       1
Name: count, Length: 958, dtype: int64

In [43]:
train_df[train_df.income_status=="Over Median"]["income"].value_counts()

income
0       354
1800     18
2000     15
1500     14
2300     14
       ... 
1093      1
2291      1
2723      1
2680      1
2373      1
Name: count, Length: 183, dtype: int64

In [44]:
train_df[train_df.income_status=="Unknown"]["income"].value_counts()

income
0       2669
500      215
800      175
600      163
700      161
        ... 
2140       1
913        1
844        1
1155       1
999        1
Name: count, Length: 659, dtype: int64

In [47]:
# income_status가 Under Median으로 적힌 것 중 가장 큰 값 : 9916 (income 컬럼의 중위값인 500보다 크다)
np.max(train_df[train_df.income_status=="Under Median"]["income"])

9916

In [48]:
# income_status가 Over Median으로 적힌 것 중 가장 작은 값 : 0 (income 컬럼의 중위값인 500보다 작다)
np.min(train_df[train_df.income_status=="Over Median"]["income"])

0

In [49]:
train_df[train_df.income==9916]

,id,age,gender,education_status,employment_status,working_week (yearly),industry_status,occupation_status,race,hispanic_origin,martial_status,household_status,household_summary,citizenship,birth_country,birth_country (father),birth_country (mother),tax_status,gains,losses,dividends,income_status,income
4951,TRAIN_04951,40,F,Associates degree (Vocational),Children or Armed Forces,52,Education,Admin Support (include Clerical),White,Mexican (Mexicano),Married,Spouse of householder,Spouse of householder,Foreign-born (Naturalized US Citizen),Mexico,Mexico,Mexico,Married Filling Jointly both under 65 (MFJ),0,0,0,Under Median,9916


In [59]:
train_df[(train_df.income_status=="Over Median") & (train_df.income==0)].shape

(354, 23)

In [60]:
train_df[(train_df.income<=9916) & (train_df.income_status=="Over Median")].shape


(736, 23)

### 함수 생성

In [51]:
def drop_col(df):
    df_copy = df.copy()

    # household_status 컬럼 삭제
    df_copy.drop(["household_status"], axis=1, inplace=True)
    
    return df_copy

In [52]:
def apply_cols(df):
    # occupation_status 변경
    # 미국 미성년자 기준 : 18세 미만
    # 18세 미만 중 occupation_status가 Unknown인 것 확인
    # 18세 미만인데  employment_status가 Children or Armed Forces
    # - employment_status를 Children으로 바꿔도 될 듯
    # - industry_status도 Children으로 변경
    # occupation_status이 Unknown이면 children으로 변경
    
    df.loc[(df.age<18) 
           & (df.occupation_status=="Unknown") 
           & (df.employment_status=="Children or Armed Forces"), 
           ["employment_status", "occupation_status"]] = ["Children",  "Children"]
    
    # 미국 군인 (준장 이하) 일반적으로 62세 정년
    # 이 사람들 중 employment_status이 Children or Armed Forces인 사람들은 어디에도 해당하지 않음
    # employment_status → Not Working으로 변경
    df.loc[(df.age>=62) 
           & (df.occupation_status=="Unknown") 
           & (df.employment_status=="Children or Armed Forces"),
           ["employment_status", "industry_status", "occupation_status"]] = ["Not Working", "Not in universe", "Not Working"]

    

    # 18세 이상 62세 미만인데 occupation_status가 Unknown
    # - industry_status : 전부 Not in universe or children
    # - income : 전부 0
    # - working_week (yearly) : 0이 많긴해도 다양함
    # employment_status가 Children or Armed Forces 인 사람들
    # - employment_status, occupation_status : Not Working 으로 변경

    df.loc[(df.age>=18) 
           & (df.age<62) 
           & (df.occupation_status=="Unknown") 
           & (df.employment_status=="Children or Armed Forces"), 
           ["employment_status", "industry_status", "occupation_status"]] = ["Not Working", "Not in universe", "Not Working"]
    
    # 18세 이상 62세 미만, occupation_status가 Unknown이 아니고, employment_status가 children or armed forces
    # → employment_status, industry_status, occupation_status : Armed Forces로 변경
    
    df.loc[(df.age>=18) 
           & (df.age<62) 
           & (df.occupation_status!="Unknown") 
           & (df.employment_status=="Children or Armed Forces"), 
           ["employment_status", "industry_status", "occupation_status"]] = ["Armed Forces", "Armed Forces", "Armed Forces"]

    
    # industry_status == "Not in universe or children" 인 사람 중, age<18이면 Children으로 변경
    # industry_status == "Not in universe or children" 인 사람 중, age>=18이면 Not in universe로 변경
    
    df.loc[(df.age<18) & (df.industry_status=="Not in universe or children"), "industry_status"] = "Children"
    df.loc[(df.age>=18) & (df.industry_status=="Not in universe or children"), "industry_status"] = "Not in universe"

    return df

In [53]:
prac = drop_col(train_df)
prac.head(3)

,id,age,gender,education_status,employment_status,working_week (yearly),industry_status,occupation_status,race,hispanic_origin,martial_status,household_summary,citizenship,birth_country,birth_country (father),birth_country (mother),tax_status,gains,losses,dividends,income_status,income
0,TRAIN_00000,63,M,Middle (7-8),Full-Time,4,Social Services,Services,White,All other,Married,Householder,Native,US,US,US,Nonfiler,0,0,0,Unknown,425
1,TRAIN_00001,37,M,Associates degree (Vocational),Full-Time,52,Entertainment,Services,White,All other,Separated,Householder,Native,US,US,US,Single,0,0,0,Under Median,0
2,TRAIN_00002,58,F,High graduate,Full-Time,52,Manufacturing (Non-durable),Admin Support (include Clerical),Black,All other,Married,Householder,Native,US,US,US,Married Filling Jointly both under 65 (MFJ),3411,0,0,Under Median,860


In [54]:
apply_cols(prac)["employment_status"].value_counts()

employment_status
Armed Forces                     7085
Full-Time                        6588
Not Working                      2387
Children                         2323
Choice Part-Time                  593
Children or Armed Forces          568
Part-Time (Usually Part-Time)     158
Seeking Full-Time                 121
Part-Time (Usually Full-Time)     100
Seeking Part-Time                  77
Name: count, dtype: int64

In [55]:
apply_cols(prac)["industry_status"].value_counts()

industry_status
Armed Forces                                    7086
Children                                        2464
Not in universe                                 2224
Retail                                          1815
Manufacturing (Durable)                          766
Manufacturing (Non-durable)                      626
Education                                        527
Business & Repair                                450
Medical (except Hospitals)                       449
Hospitals                                        432
Construction                                     427
Finance Insurance & Real Estate                  384
Transportation                                   367
Public Administration                            355
Other professional services                      281
Wholesale                                        242
Personal Services (except Private Household)     236
Social Services                                  191
Entertainment                 

In [56]:
apply_cols(prac)["occupation_status"].value_counts()

occupation_status
Armed Forces                        7086
Children                            2323
Admin Support (include Clerical)    1411
Services                            1288
Unknown                             1199
Not Working                         1166
Sales                                973
Craft & Repair                       935
Professional                         800
Machine Operators & Inspectors       687
Management                           611
Handlers/Cleaners                    471
Transportation                       385
Technicians & Support                282
Farming & Forestry & Fishing         170
Protective Services                  146
Private Household Services            67
Name: count, dtype: int64